# Capstone Project - The Battle of the Neighborhoods (Final project notebook)

#                                                                          New Cineplex in London

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

The objective of this project is to analyse and select the best location in the city of London, United Kingdom to open a new Cineplex. Specifically, this report would be targeted to Propert developers or stakeholders interested in opening an **Cineplex** in **London**, United Kingdom.

Since there are many Cineplex in London we will try to detect **locations that are not crowded with Cineplex**. Primarily we will be focusing on the crowded residential areas where there are no **Cineplex** 

Using data science methodology and machine learning techniques like clustering, this project aims to provide solutions to answer the business question: In a multi-cultural city like London, if a property developer or a stakeholder is looking to open a new Cineplex, where would you recommend, they open it?

We will use our data science power to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that the best location can be chosen by the proprty developers.

## Data

Based on definition of our problem, factor that will influence our decision are:
* Number of existing Cineplex in the neighborhood
* Most crowded areas in the neighborhood
* distance of neighborhood from city center

Following data sources will be needed to extract/generate the required information:
* List of neighborhoods in London. This defines the scope of this project which is confined to the city of London.
* Latitude and Longitude coordinates of those neighbourhoods. This is required to plot the map and get the venue data.
* Venue data, particularly data related to Cineplex. We will use this data to perform clustering on neighbourhoods.

## 1. Import libraries

In [1]:
!pip install folium
!pip install geocoder

     |████████████████████████████████| 92kB 306kB/s eta 0:00:01
     |████████████████████████████████| 102kB 339kB/s ta 0:00:01


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


## 2.Scrap data from Wikipedia page into a DataFrame

In [3]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_areas_of_London")
soup = BeautifulSoup(data.content, 'html.parser')
tb = soup.find('table', class_='wikitable')
links = tb.findAll('a')
areas = []
for link in links:
    areas.append(link.get('title'))

In [4]:
df = pd.DataFrame()
df['Areas'] = areas

In [5]:
df = df.drop_duplicates(keep="first")
df = df.drop(df.index[1])

In [6]:
df.reset_index(inplace = True)

In [7]:
df.drop(['index'], axis=1,inplace=True)
df.head()

,Areas
0,Abbey Wood
1,"Acton, London"
2,"Addington, London"
3,Addiscombe
4,"Albany Park, Bexley"


## 3. Get the geographical coordinates

In [8]:
def get_latlng(Areas):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(Areas))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Areas"].tolist() ]

In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [14]:
# check the neighborhoods and the coordinates
print(df.shape)
df.head()

(531, 3)


,Areas,Latitude,Longitude
0,Abbey Wood,51.492450,0.121270
1,"Acton, London",51.513240,-0.267460
2,"Addington, London",51.428124,-0.044685
3,Addiscombe,51.472745,-0.203324
4,"Albany Park, Bexley",51.435700,0.125880


In [15]:
# save the DataFrame as CSV file
df.to_csv("London_df.csv", index=False)

## 4. Create a map of London with neighborhoods superimposed on top

In [16]:
# get the coordinates of London
address = 'London, United Kingdom'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London, United Kingdom {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London, United Kingdom 51.5073219, -0.1276474.


In [17]:
# create map of London using latitude and longitude values
map_London = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Areas']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_London)  
    
map_London

In [18]:
# save the map as HTML file
map_London.save('map_London.html')

## 5. Use the Foursquare API to explore the neighborhoods

In [19]:
# define Foursquare Credentials and Version
CLIENT_ID = 'NYUVVZZHTBPX0UOTHNXP1ZMB4WCN5VIL53QLZOLYYYIUVRXM' # your Foursquare ID
CLIENT_SECRET = 'BRK0VQ2BZOEJZHMBCC44OWGJEYT4WNAC13QJIJNFHD0YIZ1B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [20]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Areas']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(46393, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abbey Wood,51.49245,0.12127,Sainsbury's,51.492824,0.120724,Supermarket
1,Abbey Wood,51.49245,0.12127,Lesnes Abbey,51.489526,0.125839,Historic Site
2,Abbey Wood,51.49245,0.12127,Lidl,51.496152,0.118417,Supermarket
3,Abbey Wood,51.49245,0.12127,Morrisons Thamesmead,51.507276,0.105392,Supermarket
4,Abbey Wood,51.49245,0.12127,Wilko,51.505596,0.103845,Furniture / Home Store


### Let's check how many venues were returned for each neighorhood

In [22]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Abbey Wood,18,18,18,18,18,18
"Acton, London",100,100,100,100,100,100
"Addington, London",93,93,93,93,93,93
Addiscombe,100,100,100,100,100,100
"Albany Park, Bexley",40,40,40,40,40,40
Aldborough Hatch,4,4,4,4,4,4
Aldgate,100,100,100,100,100,100
Aldwych,100,100,100,100,100,100
Alperton,100,100,100,100,100,100


### Let's find out how many unique categories can be curated from all the returned venues

In [23]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 403 uniques categories.


In [24]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Supermarket', 'Historic Site', 'Furniture / Home Store',
       'Fast Food Restaurant', 'Train Station', 'Grocery Store',
       'Clothing Store', 'Campground', 'Gym / Fitness Center',
       'Warehouse Store', 'Trail', 'Café', 'Eastern European Restaurant',
       'Pub', 'Creperie', 'Hotel', 'Park', 'Brewery', 'Bakery',
       'Falafel Restaurant', 'Go Kart Track', 'Coffee Shop',
       'Middle Eastern Restaurant', 'Recreation Center',
       'Sushi Restaurant', 'Breakfast Spot', 'Gastropub', 'Wine Shop',
       'Gym', 'Japanese Restaurant', 'Hookah Bar', 'Mini Golf',
       'Portuguese Restaurant', 'Bowling Alley', 'Film Studio',
       'Fish & Chips Shop', 'Museum', 'Lebanese Restaurant', 'Office',
       'Italian Restaurant', 'Moroccan Restaurant', 'French Restaurant',
       'Convenience Store', 'Thai Restaurant', 'Indian Restaurant',
       'Pastry Shop', 'Music Venue', 'Mediterranean Restaurant',
       'Halal Restaurant', 'Chinese Restaurant'], dtype=object)

In [25]:
# check if the results contain "Movie Theatre"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

## 6. Analyze Each Neighborhood

In [27]:
# one hot encoding
Lon_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Lon_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Lon_onehot.columns[-1]] + list(Lon_onehot.columns[:-1])
Lon_onehot = Lon_onehot[fixed_columns]

print(Lon_onehot.shape)
Lon_onehot.head()

(46393, 404)


,Neighborhoods,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cave,Cemetery,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Football Field,College Quad,College Soccer Field,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Observatory,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Paintball Field,Pakistani 

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
Lon_grouped = Lon_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(Lon_grouped.shape)
Lon_grouped.head()

(531, 404)


,Neighborhoods,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cave,Cemetery,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Football Field,College Quad,College Soccer Field,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Observatory,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Paintball Field,Pakistani 

In [29]:
len(Lon_grouped[Lon_grouped["Movie Theater"] > 0])

200

### Create a new DataFrame for Cineplex(Movie Theatre) data only

In [30]:
London_Cineplex = Lon_grouped[["Neighborhoods","Movie Theater"]]

In [31]:
London_Cineplex.head()

,Neighborhoods,Movie Theater
0,Abbey Wood,0.00
1,"Acton, London",0.01
2,"Addington, London",0.00
3,Addiscombe,0.00
4,"Albany Park, Bexley",0.00


## 7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in London into 3 clusters.

In [32]:
# set number of clusters
kclusters = 3

London_clustering = London_Cineplex.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(London_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 0, 0, 0, 1, 1, 2], dtype=int32)

In [33]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
London_merged = London_Cineplex.copy()

# add clustering labels
London_merged["Cluster Labels"] = kmeans.labels_

In [34]:
London_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
London_merged.head()

,Neighborhood,Movie Theater,Cluster Labels
0,Abbey Wood,0.00,0
1,"Acton, London",0.01,2
2,"Addington, London",0.00,0
3,Addiscombe,0.00,0
4,"Albany Park, Bexley",0.00,0


In [35]:
# merge df with London_merged to add latitude/longitude for each neighborhood
London_merged = London_merged.join(df.set_index("Areas"), on="Neighborhood")

print(London_merged.shape)
London_merged.head() # check the last columns!

(531, 5)


,Neighborhood,Movie Theater,Cluster Labels,Latitude,Longitude
0,Abbey Wood,0.00,0,51.492450,0.121270
1,"Acton, London",0.01,2,51.513240,-0.267460
2,"Addington, London",0.00,0,51.428124,-0.044685
3,Addiscombe,0.00,0,51.472745,-0.203324
4,"Albany Park, Bexley",0.00,0,51.435700,0.125880


In [36]:
# sort the results by Cluster Labels
print(London_merged.shape)
London_merged.sort_values(["Cluster Labels"], inplace=True)
London_merged.head()

(531, 5)


,Neighborhood,Movie Theater,Cluster Labels,Latitude,Longitude
0,Abbey Wood,0.0,0,51.492450,0.121270
326,Noak Hill,0.0,0,51.622170,0.225800
324,"Newington, London",0.0,0,51.550949,-0.085175
323,"Newbury Park, London",0.0,0,51.519344,-0.098296
322,New Southgate,0.0,0,51.614380,-0.142700


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(London_merged['Latitude'], London_merged['Longitude'], London_merged['Neighborhood'], London_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## 8. Examine Clusters

### Cluster 0

In [39]:
London_merged.loc[London_merged['Cluster Labels'] == 0]

,Neighborhood,Movie Theater,Cluster Labels,Latitude,Longitude
0,Abbey Wood,0.0,0,51.492450,0.121270
326,Noak Hill,0.0,0,51.622170,0.225800
324,"Newington, London",0.0,0,51.550949,-0.085175
323,"Newbury Park, London",0.0,0,51.519344,-0.098296
322,New Southgate,0.0,0,51.614380,-0.142700
321,New Malden,0.0,0,51.400904,-0.244971
320,New Eltham,0.0,0,51.433530,0.063780
319,New Cross,0.0,0,51.474890,-0.040380
318,New Barnet,0.0,0,51.627294,-0.253759
317,New Addington,0.0,0,51.428124,-0.044685


### Cluster 1

In [40]:
London_merged.loc[London_merged['Cluster Labels'] == 1]

,Neighborhood,Movie Theater,Cluster Labels,Latitude,Longitude
48,"Blackfriars, London",0.020000,1,51.512040,-0.103560
102,Cockfosters,0.045455,1,51.634541,-0.205739
511,Westminster,0.020000,1,51.500080,-0.128020
523,"Woodlands, London",0.020000,1,51.558750,-0.178370
53,"Botany Bay, London",0.020000,1,51.506420,-0.127210
367,Ponders End,0.020000,1,51.645487,-0.046534
460,Tokyngton,0.020000,1,51.506420,-0.127210
514,Whitechapel,0.020000,1,51.519170,-0.059660
454,"Temple, London",0.020000,1,51.513679,-0.110885
46,Biggin Hill,0.021978,1,51.416055,-0.104143


### Cluster 2

In [41]:
London_merged.loc[London_merged['Cluster Labels'] == 2]

,Neighborhood,Movie Theater,Cluster Labels,Latitude,Longitude
254,Islington,0.010000,2,51.532790,-0.106140
249,Hoxton,0.010000,2,51.537920,-0.099830
33,"Bedford Park, London",0.010000,2,51.519060,-0.128950
34,Belgravia,0.010000,2,51.497050,-0.152750
251,Ilford,0.013514,2,51.563760,0.067150
36,"Belmont, Harrow",0.010000,2,51.543487,-0.213975
247,Hornsey,0.010000,2,51.581700,-0.120930
198,Hackney Central,0.010000,2,51.547130,-0.055100
172,Fitzrovia,0.010000,2,51.518730,-0.137370
39,"Belvedere, London",0.010000,2,51.548630,-0.008210


## Observations:

Our Analysis shows that although there aree great number of Movie Theatres in London (~200 in our initial area of interest around London). Looking at the clusters the maximum number of Cineplex's are present in Cluster 1, very small number of Cineplex's are present in Cluster 2 and absolutely no Cineplex's are present in Cluster 0. Building a Movie Theater in any one of the locations in Cluster 0 will be a right decision. Examining the clusters more Movie Theatres are present in Central London near Thames and northern part of Central London. So opening a Movie Theatre here will not be the right choice.

The south eastern part of London such as Lewisham, Catford, Eltham., etc does not have any Movie Theatres nearby. Opening a Cineplex in any one of these will definitely attract more customers.

Also closely analysing the areas near Walthamstow, West Hackney., etc there are few number of Movie Theatres. These areas could also be the right place for opening a new Cineplex as there are few Movie Theatres in the nearby areas.

So to conclude on a shorter note, areas near Central London such as Marylebone, Kensington, Westminster, Waterloo are not the right place to open a Cineplex as there are more no.of movie theatres around. Highly recommended places to open a Cineplex are places in South East London such as Lewisham, Catford, Eltham., etc. Also, areas near North London such as Walthamstow, West Hackney., etc are quite suitable places to open a new Movie Theatre.


# Conclusion

Purpose of this project was to identify London areas with low number of Movie Theatres in order to aid stakeholders or Property developers in narrowing down the search for optimal location for a new Cineplex. By Calculating Movie Theatre density distribution from Foursquare data we have first identified the areas that have movie theatres. Then these identified locations were clustered based on the density of the Movie theatres present. Clusters gave the visualization of the density of Cineplex which gave the insight to places which has very less or absolutely no movie theatres. These addresses are chosen for optimal locations to build a new Cinplex which will then be used as starting point for final exploration by Stakeholders or Property developers.

Final decision on optimal Cineplex location will be made by Property developers based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like total population of each location (more number of people preferred), proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.